In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [2]:
# Define data directories
train_dir = 'data/train/'
valid_dir = 'data/valid/'
test_dir = 'data/test/'

# Define train transforms to augment data
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomRotation(30),
                                       transforms.ToTensor(),
#                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

# Define test/validatoin transforms without augmenting
test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor()])

# Load in the data
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# Define loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [3]:
train_data.class_to_idx

{'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}

In [4]:
# In order to handle class imbalance let's count examples belongs to each class
mela_count = 0
nev_count = 0
sebo_count = 0

for img, label in train_data.imgs:
    if label == 0:
        mela_count += 1
    if label == 1:
        nev_count += 1
    if label == 2:
        sebo_count += 1

In [5]:
# Download the pretrained densenet121 model
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [6]:
# We will freeze params for feature detection part and use it as it is
# We will only retrain the classifier of the model
print(model.classifier.in_features) 
print(model.classifier.out_features)

1024
1000


In [7]:
# Freeze training for all "features" layers
for param in model.features.parameters():
    param.requires_grad = False

In [8]:
import torch.nn as nn

n_inputs = model.classifier.in_features

last_layer = nn.Linear(n_inputs, 3)

model.classifier = last_layer

model.cuda()

# check to see that your last layer produces the expected number of outputs
print(model.classifier.out_features)

3


In [9]:
# Manually init weights to penilize the class #1
class_weights = torch.FloatTensor([1 / mela_count, 1 / nev_count, 1 / sebo_count]).cuda()

In [10]:
# Define loss function and optimizer
import torch.optim as optim
# Start with the custom weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion.cuda()

CrossEntropyLoss()

In [11]:
# Define the train method with customizable parameters
import time
def train(n_epochs, model, optimizer, criterion, save_path):
    valid_loss_min = np.Inf
    valid_accuracy = 0
    print_every = 10
    for epoch in range(n_epochs):
        t0 = time.time()
        train_loss = 0
        valid_loss = 0
        
        # Train the model
        model.train()
        for image, label in train_loader:
            image, label = image.cuda(), label.cuda()
            # Zero out the gradients
            optimizer.zero_grad()
            
            # Forward pass
            prediction = model(image)
            
            # Calculate the loss
            loss = criterion(prediction, label)
            
            # Calculate the gradients
            loss.backward()
            
            # Update the weights
            optimizer.step()
            
            # Update the train_loss
            train_loss += loss.item()
        
        # Validate the model
        model.eval()
        for image, label in valid_loader:
            image, label = image.cuda(), label.cuda()
            # Predict the image
            prediction = model(image)
            
            # Calculate the loss
            loss = criterion(prediction, label)
            
            # Update the validation loss
            valid_loss += loss.item()
            
            # Calculate the accuracy
            top_p, top_class = prediction.topk(1, dim=1)
            
            equals = top_class == label.view(*top_class.shape)
            
            valid_accuracy = torch.mean(equals.type(torch.FloatTensor))
            
            # print training/validation statistics 
        print("Epoch: {}\t Training Loss: {}\t Validation Loss: {}\t Validation Accuracy: {}% \n".format(epoch,
            train_loss/len(train_loader), valid_loss/len(valid_loader), valid_accuracy.item()*100))
            
            # Save the model if validation loss decreased
        if valid_loss <= valid_loss_min:
            print("Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(valid_loss_min,
                     valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        t1 = time.time()
        exc_time = t1-t0
        print("Epoch: {} Time: {}".format(epoch, exc_time))
    return model

In [12]:
model = train(5, model, optimizer, criterion, 'model.pt')

Epoch: 0	 Training Loss: 1.0377843957394361	 Validation Loss: 0.9059168100357056	 Validation Accuracy: 63.63636255264282% 

Validation loss decreased (   inf ===> 2.717750). Saving the model...
Epoch: 0 Time: 318.3393692970276
Epoch: 1	 Training Loss: 0.9254585858434439	 Validation Loss: 0.8550644318262736	 Validation Accuracy: 50.0% 

Validation loss decreased (2.717750 ===> 2.565193). Saving the model...
Epoch: 1 Time: 286.95943880081177
Epoch: 2	 Training Loss: 0.887622682377696	 Validation Loss: 0.8958720366160074	 Validation Accuracy: 50.0% 

Epoch: 2 Time: 289.4317169189453
Epoch: 3	 Training Loss: 0.8649337571114302	 Validation Loss: 0.7901123960812887	 Validation Accuracy: 63.63636255264282% 

Validation loss decreased (2.565193 ===> 2.370337). Saving the model...
Epoch: 3 Time: 284.424391746521
Epoch: 4	 Training Loss: 0.8384692389518023	 Validation Loss: 0.8230821092923483	 Validation Accuracy: 59.090906381607056% 

Epoch: 4 Time: 287.6356523036957


In [13]:
# show the confusion matrix on test data
nb_classes = 3

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.cuda()
        classes = classes.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

tensor([[ 74.,  13.,  30.],
        [140., 159.,  94.],
        [ 13.,   5.,  72.]])


In [13]:
# Without weight penilization
# print(confusion_matrix.diag()/confusion_matrix.sum(1))

tensor([0.1026, 0.9466, 0.1444])


In [14]:
# Let's see the correct preds by percentage
print(confusion_matrix.diag()/confusion_matrix.sum(1))

tensor([0.6325, 0.4046, 0.8000])


In [15]:
# Check the accuracy on Test Data
test_loss = 0
accuracy = 0
model.eval()
for images, labels in test_loader:
    images, labels = images.cuda(), labels.cuda()
    
    ps = model(images)
    
    loss = criterion(ps, labels)
    
    test_loss += loss.item()
    
    top_p, top_class = ps.topk(1, dim=1)
            
    equals = top_class == labels.view(*top_class.shape)
            
    accuracy = torch.mean(equals.type(torch.FloatTensor))
    
    print("Test Loss: {}\tTest Accuracy: {}".format(test_loss/len(test_loader), accuracy.item()*100))

Test Loss: 0.09643087983131408	Test Accuracy: 39.0625
Test Loss: 0.18565444946289061	Test Accuracy: 53.125
Test Loss: 0.2666580319404602	Test Accuracy: 51.5625
Test Loss: 0.35518312454223633	Test Accuracy: 46.875
Test Loss: 0.4491594612598419	Test Accuracy: 53.125
Test Loss: 0.5247509002685546	Test Accuracy: 57.8125
Test Loss: 0.6108143985271454	Test Accuracy: 51.5625
Test Loss: 0.7028481304645539	Test Accuracy: 51.5625
Test Loss: 0.7929889738559723	Test Accuracy: 43.75
Test Loss: 0.878148102760315	Test Accuracy: 54.16666865348816


<h3>Results</h3>
- Even though we have a lower overall accuracy(50.2%), from the confusion matrix we can see that we successfully detected 63% of the melanoma dataset.